In [42]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count

import pickle
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from torch.nn import CrossEntropyLoss, MSELoss

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

from multiprocessing import Pool, cpu_count
# from tools import *
# import convert_examples_to_features


# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
train_data = pd.read_csv("train.tsv", sep='\t', header=None)

In [3]:
train_data.columns = ['claimID', 'claim', 'label', 
                      'claimURL', 'reason', 'categories', 
                      'speaker', 'checker', 'tags', 'articleTitle', 'publishDate', 'claimDate', 'entities']

In [4]:
train_data.head(1)

,claimID,claim,label,claimURL,reason,categories,speaker,checker,tags,articleTitle,publishDate,claimDate,entities
0,pomt-03627,Six out of 10 of the highest unemployment rate...,half-true,/ohio/statements/2013/may/06/chris-redfern/ohi...,When a couple of Statehouse Republicans prepar...,None,Chris Redfern,None,None,None,2013-05-06T06:00:00,2013-04-30,['None']


In [5]:
pd.isnull(train_data['claimID']).sum()

0

In [6]:
pd.isnull(train_data['claim']).sum()

1

In [17]:
claim = train_data['claim']
label = train_data['label']
reason = train_data['reason']

type(label)

pandas.core.series.Series

In [18]:
# exploring the types of labels
# we have 117 different types of labels here

# QUES - what are 0, 1, 2, 3, 4?

print(label.value_counts())

false                              5537
mostly true                        2466
mostly false                       2454
true                               2442
half-true                          2428
0                                  1957
pants on fire!                     1287
fiction!                           1246
none                                933
truth!                              621
mixture                             555
promise kept                        441
unproven                            431
facebook scams                      384
1                                   253
true messages                       242
promise broken                      218
compromise                          185
unproven!                           176
bogus warning                       176
in-between                          175
truth! & fiction!                   137
verdict: true                       125
verdict: false                      124
miscaptioned                        122


In [23]:
label_df = label.to_frame()

# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column
label_df['label_num'] = labelencoder.fit_transform(label_df['label'])
label_df

,label,label_num
0,half-true,51
1,true,96
2,fiction!,45
3,false,43
4,true,96
5,mostly false,67
6,false,43
7,determination: mostly true,30
8,truth!,98
9,full flop,47


In [33]:
claim_df = claim.to_frame()

following this approach
https://medium.com/swlh/a-simple-guide-on-using-bert-for-text-classification-bbf041ac8d04

In [34]:
train_df_bert = pd.DataFrame({
    'id':range(len(label_df)),
    'label':label_df['label_num'],
    'alpha':['a']*label_df.shape[0],
    'text': claim_df['claim']})

In [35]:
train_df_bert.head()

,id,label,alpha,text
0,0,51,a,Six out of 10 of the highest unemployment rate...
1,1,96,a,No Democratic campaign for (Fla.) governor has...
2,2,45,a,Forward an email for Jasmine
3,3,43,a,Pope Francis endorsed Donald Trump for president.
4,4,96,a,Says Ron Paul insisted FEMA should be shut down.


In [37]:
# converting to tsv and saving
train_df_bert.to_csv('train_wip.tsv', sep='\t', index=False, header=False)

In [ ]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "data/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bert-base-cased'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'yelp'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'reports/{TASK_NAME}_evaluation_report/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = 'cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

### Stuff to try later

In [16]:
# training data later 

def create_dataloader_training(features):
    # The next lines are taken from the example at https://github.com/huggingface/transformers/blob/0cb163865a4c761c226b151283309eedb2b1ca4d/transformers/data/processors/glue.py#L30
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

    class_counts = np.bincount(all_labels)
    class_sample_freq = 1 / class_counts
    weights = [class_sample_freq[label] for label in all_labels]
    print(class_counts)
    print(class_sample_freq)
    num_samples = round(class_counts[1] * 2).item()  # .item to convert to native int
    print(f'Num samples: {num_samples}')
    sampler = WeightedRandomSampler(weights, num_samples=num_samples,
                                    replacement=True)  # we want to use all positive instances and use equally as many negative instances. This should now generally happen by chance
    dataloader = DataLoader(dataset, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [23]:
len(claim)

27871

In [24]:
claim[2]

'Forward an email for Jasmine'

In [28]:
m = []
for i in (0,3):
    t = tokenizer.tokenize(claim[i])
    m.append(t)
    
print(m)

[['six', 'out', 'of', '10', 'of', 'the', 'highest', 'unemployment', 'rates', 'are', 'also', 'in', 'so', '-', 'called', 'right', 'to', 'work', 'states', '.'], ['pope', 'francis', 'endorsed', 'donald', 'trump', 'for', 'president', '.']]


In [29]:
# trying out BERT tokenizer - making a list of lists for the claim


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
# tokenizes claim from 

tokenized_claim = []

for i in range(len(claim)):
    t = tokenizer.tokenize(claim[i])
    tokenized_claim.append(t)
    

TypeError: 'float' object is not iterable